# Model Output Notebook

<img style="float:center;" src="https://arcticexpansion.vse.gmu.edu/sites/arcticexpansion.vsnet.gmu.edu/files/images/header5d2.png" width=600px>

### ADCIRC-SWAN Output


### Initialize Libraries

In [1]:
import netCDF4 as nc4;        import pandas as pd
import pathlib as pl;         import geopandas as gpd
import numpy as np;           import xarray as xr

from shapely import Polygon,Point,MultiPoint,LineString,MultiLineString;import shapely.vectorized
from sklearn.neighbors import BallTree
from scipy.stats import linregress

source $HOME/miniforge3/bin/activate

salloc --ntasks=5 --nodes=1 --partition=normal --time=10:00:00

### Defined Functions

In [2]:

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

def point_lookup(model_lat:np.array, model_lon:np.array, satellite_lat:np.array, satellite_lon:np.array):
    tree = BallTree(np.deg2rad(np.c_[model_lat,model_lon]), metric='haversine')
    distances, indices = tree.query(np.deg2rad(np.c_[satellite_lat, satellite_lon]), k = 1)
    return distances*6371,indices


#### Data for this exercise can be found here
https://doi.org/10.17603/ds2-h0fw-2p96

Download the swan_HS.63.nc from one of the 4 folders

---

In [3]:
root = pl.Path('/groups/ORC-CLIMATE/fhrl_repo/Arctic_Database/Raw_DATA')
outdir = pl.Path('/groups/ORC-CLIMATE/fhrl_repo/Arctic_Database/Processed_DATA')

In [4]:
gdf = gpd.read_file('/groups/ORC-CLIMATE/fhrl_repo/Arctic_Database/arctic_shapefiles/comm4process/nodes4communities.shp')

In [5]:
comm_lats = gdf.geometry.y.values
comm_lons = gdf.geometry.x.values
comm_names = gdf["community"].astype(str).values  # Adjust to your column name
node_ids = gdf["node_ids"].values.astype(int)         # Adjust to your node column name
max_name_len = max(len(name) for name in comm_names)

In [6]:
year = 2024
output_dir = root / str(year) / "outputs"

outdir.mkdir(parents=True, exist_ok=True)
file_var_map = {
    "zeta": "fort.63.nc",
}
'''
file_var_map = {
    "zeta": "fort.63.nc",
    "u-vel": "fort.64.nc",
    "v-vel": "fort.64.nc",
    "pressure": "fort.73.nc",
    "windx": "fort.74.nc",
    "windy": "fort.74.nc",
    "iceaf": "fort.93.nc",
    "swan_HS": "swan_HS.63.nc",
    "swan_TPS": "swan_TPS.63.nc",
    "swan_DIR": "swan_DIR.63.nc"
}
'''
from collections import defaultdict
file_to_vars = defaultdict(list)
for var, fname in file_var_map.items():
    file_to_vars[fname].append(var)

In [ ]:
data_by_var = {var: [] for var in file_var_map}
time_list = []

for fname, var_list in file_to_vars.items():
    file_path = output_dir / fname
    if not file_path.exists():
        print(f"⚠️ File not found: {file_path}")
        continue

    with nc4.Dataset(file_path) as ds:
        if "time" in ds.variables:
            time = ds.variables["time"][:]
            time_list.append(time)

        for var in var_list:
            nc_var = var if var in ds.variables else var.replace("-", "_")
            if nc_var in ds.variables:
                var_data = ds.variables[nc_var][:, node_ids]
                data_by_var[var].append(var_data)
            else:
                print(f"⚠️ Variable '{nc_var}' not found in {fname}")

In [ ]:
time_all = np.concatenate(time_list) if time_list else np.array([])

for var in data_by_var:
    if data_by_var[var]:  # only concatenate if data was found
        data_by_var[var] = np.concatenate(data_by_var[var], axis=0)
    else:
        data_by_var[var] = np.full((len(time_all), len(node_ids)), np.nan)

out_path = outdir / f"{year}.nc"
with nc4.Dataset(out_path, "w", format="NETCDF4") as ds_out:
    nt, nn = len(time_all), len(node_ids)

    # Dimensions
    ds_out.createDimension("time", nt)
    ds_out.createDimension("node", nn)
    ds_out.createDimension("name_strlen", max_name_len)

    # Coordinates and metadata
    tvar = ds_out.createVariable("time", "f8", ("time",))
    tvar[:] = time_all
    tvar.units = "seconds since 1970-01-01 00:00:00"
    tvar.calendar = "standard"

    lat = ds_out.createVariable("lat", "f4", ("node",)); lat[:] = comm_lats
    lon = ds_out.createVariable("lon", "f4", ("node",)); lon[:] = comm_lons

    name_array = np.array([list(n.ljust(max_name_len)) for n in comm_names], dtype="S1")
    name_var = ds_out.createVariable("community", "S1", ("node", "name_strlen"))
    name_var[:, :] = name_array

    # Output variables
    for var in data_by_var:
        v = ds_out.createVariable(var, "f4", ("time", "node"), zlib=True)
        v[:, :] = data_by_var[var]

print(f"✅ Extracted and saved to {out_path}")

In [ ]:
files = list((root / '2024'/'outputs').glob('*.nc'))
years = ['2023','2022','2021','2020','2019','2018',
         '2017','2016','2015','2014','2013','2012',
         '2011','2010','2009','2008','2007','2006','2005']

### Initialize path and read netcdf file

In [12]:


ncfile = nc4.Dataset(root / '2024' /'outputs' / 'fort.74.nc')

In [13]:
ncfile

<class 'netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    _FillValue: -99999.0
    model: ADCIRC
    version: noaa.stofs.2d.glo.v1.1.0r2-31-ga9ff86d
    grid_type: Triangular
    description: OceanMesh2D
    agrid: OceanMesh2D
    rundes: OceanMesh2D
    runid: simtest
    title: OceanMesh2D
    institution: Notre Dame CHL
    source: OceanMesh2D
    history: History: None
    references: https://github.com/CHLNDDEV/OceanMesh2D/
    comments: Comments: None
    host: Host: Name
    convention: Metric, MSL
    Conventions: UGRID-0.9.0
    contact: name@instit.edu
    creation_date: 2025-02-21 11:33:32 -05:00
    modification_date: 2025-02-21 11:33:32 -05:00
    fort.15: ==== Input File Parameters (below) ====
    dt: 3.0
    ihot: 0
    ics: 2
    nolibf: 1
    nolifa: 2
    nolica: 1
    nolicat: 1
    nwp: 2
    ncor: 1
    ntip: 2
    nws: 12
    nramp: 1
    tau0: -3.0
    statim: 0.0
    reftim: 0.0
    rnday: 382.0
    dramp: 2.0
    a00: 0.4
    b00